(a) Simulate the performance of Alamouti two-antenna transmit diversity. Create a random
stream of, say, 1000 QPSK symbols $(±1 ± j)/√2$. Generate a sequence of channel matrices by moving the receiver (or transmitter) through a multipath environment slowly enough that the slow fading assumption holds, but fast enough that the channel is random over the whole simulation. Generate complex Gaussian noise with variance $σ^2_η$ to simulate noise at the receiver. Assuming the receiver knows the channel perfectly, detect the symbols using the Alamouti scheme. Compute the symbol error rate as the ratio of incorrectly decoded symbols to the total number of symbols. Repeat for different values of SNR, and plot the symbol error rate on a log scale as a function of SNR in dB.

(b) Repeat for the case of no transmit diversity. Overlay the results on the same plot. To make this a fair comparison, the transmit symbols
should not be divided by √2, so that the same total transmit power as in the Alamouti case is supplied to a single antenna.